In [2]:
from cobra.io import read_sbml_model, write_sbml_model
from mewpy.cobra.util import add_enzyme_constraints
from mewpy.simulation import get_simulator
from mewpy.simulation.environment import Environment as Environment
from mewpy.util.request import retreive_gene,retreive_protein
import json

In [3]:
model = read_sbml_model("models/non-ec/")
sim = get_simulator(model)
sim.set_objective("BIOMASS")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-01


No objective coefficients in model. Unclear what should be optimized


In [4]:
sim.simulate()

objective: 28.21575880575294
Status: OPTIMAL
Constraints: OrderedDict()
Method:SimulationMethod.FBA

In [1]:
for rxn in sim.reactions:
    print(rxn,sim.get_gpr(rxn))
    
'''
for gene in sim.get_gpr:
    info = retreive_protein(gene,"Escherichia coli")
    print(info)
'''

NameError: name 'sim' is not defined

In [15]:
for rxn in sim.reactions:
    print(rxn,sim.get_substrates(rxn))
    

GLUPRT {'gln__L_c': -1.0, 'h2o_c': -1.0, 'prpp_c': -1.0}
PDE1 {'camp_c': -1.0, 'h2o_c': -1.0}
EX_g3ps_e {'g3ps_e': -1.0}
UAG2E {'uacgam_c': -1.0}
PUNP6 {'din_c': -1.0, 'pi_c': -1.0}
PUNP7 {'xtsn_c': -1.0, 'pi_c': -1.0}
PUNP4 {'dgsn_c': -1.0, 'pi_c': -1.0}
HYXNtex {'hxan_e': -1.0}
PUNP5 {'ins_c': -1.0, 'pi_c': -1.0}
HCYSMT {'amet_c': -1.0, 'hcys__L_c': -1.0}
ACACtex {'acac_e': -1.0}
METTRS {'atp_c': -1.0, 'trnamet_c': -1.0, 'met__L_c': -1.0}
EX_cynt_e {'cynt_e': -1.0}
ADOCBIK {'atp_c': -1.0, 'adocbi_c': -1.0}
GLUCYS {'glu__L_c': -1.0, 'cys__L_c': -1.0, 'atp_c': -1.0}
PLIPA2E141pp {'h2o_p': -1.0, 'pe141_p': -1.0}
PE141abcpp {'atp_c': -1.0, 'pe141_c': -1.0, 'h2o_c': -1.0}
_DCYTtex {'dcyt_e': -1.0}
PAPA161pp {'h2o_p': -1.0, 'pa161_p': -1.0}
XANtpp {'xan_p': -1.0}
EX_acac_e {'acac_e': -1.0}
MLTGY2pp {'murein4p3p_p': -1.0}
PSD160 {'ps160_c': -1.0, 'h_c': -1.0}
3HAD60 {'3hhexACP_c': -1.0}
PUNP2 {'dad_2_c': -1.0, 'pi_c': -1.0}
PUNP3 {'gsn_c': -1.0, 'pi_c': -1.0}
PUNP1 {'pi_c': -1.0, 'adn_c': -

### Retreive MW infos

The protein IDs and MW (Da) are obtained from UniProt.

Note: does not retreive Kcats from BRENDA. All catalytic rates are equal to 100000/s 

In [10]:
data_all = dict()
try:
    with open(sim.id+".json", 'r') as f:
        data = json.load(f)
except:
    data = dict()
    for gene in sim.genes:
        try:
            info = retreive_gene(gene,"Escherichia coli")
            # set the inverse kcat to a low value
            data[gene]={'protein':info['protein'],'mw':info['mw']*0.001,'kcat':1e-5}
            data_all[gene]=info
        except:
            data[gene]={'protein': gene[len(sim._g_prefix):], 'mw': 0, 'kcat': 0}
    with open(sim.id+".json", "w") as f:
        json.dump(data,f)

In [11]:
print("Genes with missing data:")
for k,v in data.items():
    if v['mw']==0:
        print(k)

Genes with missing data:
b1692


### Build the ec model

In [6]:
print(data)
ec_sim = add_enzyme_constraints(sim, data=data)

{'b0002': {'protein': 'P00561', 'mw': 89.12, 'kcat': 1e-05}, 'b0003': {'protein': 'P00547', 'mw': 33.624, 'kcat': 1e-05}, 'b0004': {'protein': 'P00934', 'mw': 47.114000000000004, 'kcat': 1e-05}, 'b0007': {'protein': 'P30143', 'mw': 51.663000000000004, 'kcat': 1e-05}, 'b0008': {'protein': 'P0A870', 'mw': 35.219, 'kcat': 1e-05}, 'b0019': {'protein': 'P13738', 'mw': 41.356, 'kcat': 1e-05}, 'b0025': {'protein': 'P0AG40', 'mw': 34.734, 'kcat': 1e-05}, 'b0026': {'protein': 'P00956', 'mw': 104.297, 'kcat': 1e-05}, 'b0029': {'protein': 'P62623', 'mw': 34.775, 'kcat': 1e-05}, 'b0030': {'protein': 'P22564', 'mw': 32.561, 'kcat': 1e-05}, 'b0031': {'protein': 'P04036', 'mw': 28.757, 'kcat': 1e-05}, 'b0032': {'protein': 'P0A6F1', 'mw': 41.431, 'kcat': 1e-05}, 'b0033': {'protein': 'P00968', 'mw': 117.842, 'kcat': 1e-05}, 'b0036': {'protein': 'P31551', 'mw': 28.19, 'kcat': 1e-05}, 'b0037': {'protein': 'P31552', 'mw': 58.559000000000005, 'kcat': 1e-05}, 'b0038': {'protein': 'P31572', 'mw': 45.127, 'kc

TypeError: add_enzyme_constraints() got an unexpected keyword argument 'data'

### Protein pool usage

In [ ]:
res = ec_sim.simulate()
print(res)

In [ ]:
res.find('pool')

### Save the model

In [ ]:
write_sbml_model(ec_sim.model,"eciAF1260.xml")

In [ ]:
env = Environment.complete(ec_sim, max_uptake=1000.0, inplace=False)

In [ ]:
env['prot_pool_exchange']=(0,0.8)

In [ ]:
ec_sim.simulate(constraints=env)

In [ ]:
data_all